# Truncated SVD

El algoritmo de reducción dimensional de **_Truncated SVD_** implementa una variante de la descomposición en valores singulares (SVD) en la que sólo se computan los *k* valores singulares más altos. Este *k* representará, por tanto, el número de dimensiones en las que desea reducirse el conjunto de datos.

Matemáticamente, _Truncated SVD_ es aplicado una matriz $X$ con $n$ muestras (filas) y $m$ características (columnas) para generar una aproximación de bajo dimensión de $X$:

$$
X \approx X_k = U_k \Sigma_k V_k^T
$$

Este proceso de factorización suele llevarse a cabo mediante dos algoritmos: [ARPACK](https://docs.scipy.org/doc/scipy/reference/tutorial/arpack.html) o [_randomized_](https://arxiv.org/abs/0909.4061).

Tras el proceso de factorización, $U_k \Sigma_k$ contendrá la representación _k_-dimensional de nuestro conjunto de datos. Igualmente, es posible transformar nuevas muestras $X^\prime$ del siguiente modo:

$$
X^\prime_k = X^\prime V_k
$$

_Truncated SVD_ es similar a PCA pero, a diferencia de este:

1. No requiere que la matriz $X$ esté estandarizada (centrada).
2. Puede ser utilizado en matrices dispersas de forma eficiente.

Este tipo de matrices dispersas son frecuentes cuando se trabaja sobre un conjunto de datos de documentos codificados mediante _Bag of Words (BOW)_ o _TF/IDF_. En esos casos, al algoritmo de _Truncated SVD_ se le conoce como _Latent Semantic Analysis (LSA)_ o _Latent Semantic Indexing (LSI)_. 

Veamos un ejemplo de la implementación de [`TruncatedSVD`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD) sobre el conjunto de datos de [_20 news group_](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html?highlight=news%20group#sklearn.datasets.fetch_20newsgroups).

Cargamos el conjunto de datos:

In [ ]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups()

Lo procesamos con TF/IDF:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer().fit(news.data)

X = vectorizer.transform(news.data)

In [ ]:
X.shape

Ejecutamos `TruncatedSVD`:

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=20, random_state=42).fit(X)

Obtenemos la transformación usando `transform`. Esta transformación se corresponde con $U_k\Sigma_k$:

In [ ]:
X_low = svd.transform(X)

In [ ]:
X_low.shape

Mediante `components_` podemos recuperar $V_k$:

In [ ]:
V = svd.components_

In [ ]:
V.shape

Esta matrices nos permiten calcular la similaridad entre dos palabras:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

vocabulary = vectorizer.vocabulary_

word_a = V.T[vocabulary['religion']].reshape(1, -1)
word_b = V.T[vocabulary['food']].reshape(1, -1)

cosine_similarity(word_a, word_b)

O entre dos documentos:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

doc_1 = 10
print('First document:\n', news.data[doc_1])

doc_2 = 500
print('\n\nSecond document:\n', news.data[doc_2])

sim = cosine_similarity(X_low[doc_1].reshape(1,-1), X_low[doc_2].reshape(1,-1))
print('\n\nSimilarity:', sim[0][0])

---

Creado por **Fernando Ortega** (fernando.ortega@upm.es)

<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png">